In [2]:
import mido as md
import numpy as np
from collections import defaultdict
import sklearn as sk

In [3]:
def generate_transition_probs(filepath):
    poops = []
    song = md.MidiFile(filepath)
    
    for track in song.tracks:
        poop = np.zeros((127, 127))
        prevnote = None
        n = np.zeros(127)
        for msg in track:
            x = vars(msg)
            if x['type'] == 'note_on' and x['velocity'] != 0:
                note = x['note']
                if prevnote is not None:
                    poop[prevnote][note] += 1
                    n[prevnote] += 1
                prevnote = note

        for i in range(127):
            if (n[i] != 0):
                poop[i] /= n[i]
        poops.append(poop)
    return poops


In [4]:
def transpose(filepath, offset):
    result = md.MidiFile()
    
    song = md.MidiFile(filepath)
    for track in song.tracks:
        newtrack = md.MidiTrack()
        result.tracks.append(newtrack)
        for msg in track:
            x = vars(msg)
            if x['type'] == 'note_on':
                note = x['note']
                note += offset
                newmessage = md.Message('note_on', note=note, velocity=x['velocity'], time=x['time'])
                newtrack.append(newmessage)
            else:
                newtrack.append(msg)
    result.save("midis/1.mid")

In [5]:
def findKeySignature(song):
    for i, track in enumerate(song.tracks):
        for msg in track:
            if msg.type == 'key_signature':
                return msg.key
            
def findTransposeOffset(filepath):
    song = md.MidiFile(filepath)
    keys = ['C', 'C#/Db', 'D', 'D#/Eb', 'E', 'F', 'F#/Gb', 'G', 'G#/Ab', 'A', 'A#/Bb', 'B']
    keySig = findKeySignature(song)
    print(keySig)
    for i in range(len(keys)):
        if keySig in keys[i]:
            offset = i
            break
    return offset

def removeZeroNotes(matrix):
    for j in (len(matrix)):
        i = len(matrix) - j - 1
        if not row.any(): #this means it's all zero
            np.delete(matrix, i, 0)
            np.delete(matrix, i, 1)
    return matrix
    
    
def findStationaryDist(matrix):
    try:
        w, v = np.linalg.eig(matrix.T)
    except LinAlgError as e:
        return None
    if 1 in w:
        return v[list(w).index(1)]
    else:
        return None


# dicking around and trying to make the left hand/right hand stuff better

In [6]:
absoluteList = defaultdict(list)
song = md.MidiFile('./midis/chopinblack.mid')
for i, track in enumerate(song.tracks):
    for msg in track:
        if msg.type == 'note_on' and msg.velocity:
            absoluteList[i].append(msg)
    
    ####THIS IS BOUND TO FUCK UP AND HAVE MULTIPLE POTENTIAL LEFT/RIGHT HAND TRACKS, OR EVEN EMPTY TRACKS
    ####I MANUALLY INDEXED FOR NOW BELOW


absoluteTime = 0
rightHandTrack = []
leftHandTrack = []
for msg in absoluteList[1]:
    rightHandTrack.append((absoluteTime, msg.note, 1))
    absoluteTime += msg.time
    
absoluteTime = 0  
for msg in absoluteList[2]:
    leftHandTrack.append((absoluteTime, msg.note, 2))
    absoluteTime += msg.time

combined = rightHandTrack + leftHandTrack

In [7]:
#now, calculate transition probabilities for R->L. 'Combined' has tuples of (time, note, hand)
notes = sorted(combined, key=lambda x: x[0])
pMatrix = np.zeros((254, 254))
prevNote = None
for note in notes:
    if prevNote:
        pMatrix[prevNote][note[1]*note[2]] += 1
    prevNote = note[1]*note[2]
        
pMatrix = sk.preprocessing.normalize(pMatrix, axis=1, norm='l1')
    
#now, given this pMatrix, need to 'step through' it - HOWEVER
#for the coordinate generated, needs to be placed in a separate track

songLength = 500
starting_note = 66

from mido import Message, MidiFile, MidiTrack

delta = 80
current_note = starting_note
current_time = 0

outfile = MidiFile()
trackRight = MidiTrack()
trackLeft = MidiTrack()
outfile.tracks.append(trackRight)
outfile.tracks.append(trackLeft)
trackRight.append(Message('program_change', program=12))
trackLeft.append(Message('program_change', program=12))

right = True
for i in range(songLength):
    if right:
        track = trackRight
    else:
        track = trackLeft
    
    track.append(Message('note_on', note = current_note, velocity = 60, time = delta))
    possibleNexts = {i:pMatrix[current_note][i] for i in range(255) if pMatrix[current_note][i] > 0}
    notes = list(possibleNexts.keys())
    probabilities = [possibleNexts[n] for n in notes]
    prev_note = current_note
    current_note = np.random.choice(notes, 1, p=probabilities)[0]
    if current_note%127 != prev_note%127:
        right = not right

outfile.save('testChopin.mid')


AttributeError: module 'sklearn' has no attribute 'preprocessing'